In [11]:
!pip install boruta


In [12]:
from boruta import BorutaPy

In [74]:
import pandas as pd
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import gradio as gr

# load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ["preg", "plas", "pres", "skin", "test", "mass", "pedi", "age", "class"]
dataset = pd.read_csv(url, names=names)

# separate features and target variable
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

# perform standard scaling on the features

scaler = StandardScaler()
X = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [75]:
dataset.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [77]:

# X_train = X_train.values
# X_test = X_test.values
y_train = y_train.values
#y_test = y_test.values

print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [78]:

# perform boruta feature selection
rfc = RandomForestClassifier(n_estimators=1000, n_jobs=-1, class_weight="balanced", max_depth=5)
boruta_selector = BorutaPy(rfc, n_estimators="auto", verbose=2, random_state=1)
boruta_selector.fit(X_train,y_train)


Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	8
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	8
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	8
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	8
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	8
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	8
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	8
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	4
Tentative: 	2
Rejected: 	2
Iteration: 	9 / 100
Confirmed: 	4
Tentative: 	2
Rejected: 	2
Iteration: 	10 / 100
Confirmed: 	4
Tentative: 	2
Rejected: 	2
Iteration: 	11 / 100
Confirmed: 	4
Tentative: 	2
Rejected: 	2
Iteration: 	12 / 100
Confirmed: 	4
Tentative: 	2
Rejected: 	2
Iteration: 	13 / 100
Confirmed: 	4
Tentative: 	2
Rejected: 	2
Iteration: 	14 / 100
Confirmed: 	4
Tentative: 	2
Rejected: 	2
Iteration: 	15 / 100
Confirmed: 	4
Tentative: 	2
Rejected: 	2
Iteration: 	16 / 100
Confirmed: 	4
Tentative: 	2
Rejected: 	2
Iteration: 	17 / 

BorutaPy(estimator=RandomForestClassifier(class_weight='balanced', max_depth=5,
                                          n_estimators=69, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x181F7267740),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x181F7267740, verbose=2)

In [79]:
selected_features = boruta_selector.support_
selected_features

array([ True,  True, False, False, False,  True,  True,  True])

In [80]:
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]

In [81]:
print(X_selected.dtype.names)
print(X_train_selected.shape)
print(X_test_selected.shape)
print(y_train.shape)
print(y_test.shape)

None
(537, 5)
(231, 5)
(537,)
(231,)


In [88]:
scaler1 = StandardScaler()
X_train_selected = scaler1.fit_transform(X_train_selected)
X_test_selected = scaler1.fit_transform(X_test_selected)
print(X_train_selected.shape)
print(y_train.shape)

(537, 5)
(537,)


In [89]:

# train the random forest classifier on the selected features
rfc = RandomForestClassifier(n_estimators=1000, n_jobs=-1, class_weight="balanced", max_depth=5)
rfc.fit(X_train_selected, y_train)


RandomForestClassifier(class_weight='balanced', max_depth=5, n_estimators=1000,
                       n_jobs=-1)

In [90]:

# define a function to make predictions using the trained model
def predict_diabetes(pregnancies, glucose, bmi, diabetes_pedigree_function, age):
    data = [[pregnancies, glucose,bmi, diabetes_pedigree_function, age]]
    data = scaler1.transform(data)
#     data_selected = data[:, selected_features]
    prediction = rfc.predict(data)[0]
    if prediction == 0:
        return "No Diabetes"
    else:
        return "Diabetes"



In [91]:

# define the input fields for Gradio
# define the input fields for Gradio
input_fields = [
    gr.inputs.Number(label="Number of Pregnancies", default=0),
    gr.inputs.Number(label="Glucose Level", default=100),
    gr.inputs.Number(label="BMI", default=30),
    gr.inputs.Number(label="Diabetes Pedigree Function", default=0.5),
    gr.inputs.Number(label="Age", default=25),
]


# define the output field for Gradio
output_field = gr.outputs.Textbox(label="Diabetes Prediction")

# launch the Gradio interface
iface = gr.Interface(fn=predict_diabetes, inputs=input_fields, outputs=output_field, title="Diabetes Predictor")
iface.launch()

C:\Users\Lenovo GEIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\inputs.py:59: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\Lenovo GEIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\Lenovo GEIN\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
